In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import pandas as pd
import torch
import numpy as np
import random
import re
import spacy
import shutil
import matplotlib.pyplot as plt
!pip install transformers
!pip install optuna
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, BertTokenizer, AutoModel, BertModel
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split, RandomSampler, SequentialSampler
import time
import optuna
from scipy.stats import spearmanr
import psycopg2
import csv
import math
import os
from sqlalchemy import create_engine
from optuna.storages import RDBStorage
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.0 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
path_base = '/content/drive/MyDrive/Tesis/STS_Benchmark/transformer_tunned_BERT/uncase_base/study_optuna/'

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget -O Stsbenchmark.tar.gz http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
shutil.unpack_archive('./Stsbenchmark.tar.gz', extract_dir='./', format='gztar')

--2023-05-08 17:39:37--  http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
Resolving ixa2.si.ehu.es (ixa2.si.ehu.es)... 158.227.106.100
Connecting to ixa2.si.ehu.es (ixa2.si.ehu.es)|158.227.106.100|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://ixa2.si.ehu.eus/stswiki/images/4/48/Stsbenchmark.tar.gz [following]
--2023-05-08 17:39:38--  http://ixa2.si.ehu.eus/stswiki/images/4/48/Stsbenchmark.tar.gz
Resolving ixa2.si.ehu.eus (ixa2.si.ehu.eus)... 158.227.106.100
Connecting to ixa2.si.ehu.eus (ixa2.si.ehu.eus)|158.227.106.100|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 409630 (400K) [application/x-gzip]
Saving to: ‘Stsbenchmark.tar.gz’

Stsbenchmark.tar.gz 100%[===================>] 400.03K  1.03MB/s    in 0.4s    

2023-05-08 17:39:38 (1.03 MB/s) - ‘Stsbenchmark.tar.gz’ saved [409630/409630]



In [ ]:
def getSTSBenchmarkSents(filename='sts-train.csv', root='stsbenchmark/', encoding='utf-8'):
  f = open(root+filename, 'r', encoding=encoding)
  s1, s2, target = [], [], []
  for line in f:
    example = re.split(r'\t+', line)
    if len(example) > 7:
      example = example[:-2]
    s2.append(example[-1])
    s1.append(example[-2])
    target.append(float(example[-3]))
  print("{} samples: {}".format(filename, len(target)))
  return s1, s2, target

In [ ]:
s1_train,s2_train,target_train = getSTSBenchmarkSents(filename='sts-train.csv')

sts-train.csv samples: 5749


In [ ]:
s1_test,s2_test,target_test= getSTSBenchmarkSents(filename='sts-test.csv')

sts-test.csv samples: 1379


In [ ]:
s1_dev,s2_dev,target_dev= getSTSBenchmarkSents(filename='sts-dev.csv')

sts-dev.csv samples: 1500


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

Device: cuda


In [ ]:
BATCH_SIZE = 32
MAX_LEN = 128
CORPUS = 'STS-B'
BERT_PATH = "bert-base-uncased"

In [ ]:
TOKENIZER = BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)

In [ ]:
def encode_sents(sents1, sents2):
    input_ids_ = []
    attention_masks_ = []
    type_ids_ = []
    for i, sent1 in enumerate(sents1):
        encoded_dict = TOKENIZER.encode_plus(
                            sent1,                      # Sentence 1 to encode.
                            sents2[i],                  # Sentence 2 to encode.
                            add_special_tokens = True,  # Add '[CLS]' and '[SEP]'
                            truncation = True,
                            max_length = MAX_LEN,       # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                      )

        # Add the encoded sentence to the list.
        input_ids_.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks_.append(encoded_dict['attention_mask'])

        type_ids_.append(encoded_dict['token_type_ids'])
    return input_ids_, attention_masks_, type_ids_

In [ ]:
def getEncodedTensors(s1, s2, labels):
    input_ids, attention_masks, type_ids = encode_sents(s1, s2)
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    type_ids = torch.cat(type_ids, dim=0)
    labels = torch.tensor(labels)
    return input_ids, attention_masks, type_ids, labels

In [ ]:
input_ids_train,attention_masks_train,type_ids_train,labels_train = getEncodedTensors(s1_train,s2_train,target_train)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
input_ids_eval, attention_masks_eval, type_ids_eval, labels_eval = getEncodedTensors(s1_dev, s2_dev, target_dev)

In [ ]:
# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(input_ids_train, attention_masks_train, type_ids_train, labels_train)
val_dataset = TensorDataset(input_ids_eval, attention_masks_eval, type_ids_eval, labels_eval)

In [ ]:
# Create the DataLoaders for our training and test sets.
# We'll take training samples in random order.
torch.manual_seed(42)
train_loader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = BATCH_SIZE # Trains with this batch size.
            #num_workers=2
        )
torch.manual_seed(42)
val_loader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = BATCH_SIZE # Evaluate with this batch size.
            #num_workers=2
        )

In [ ]:
# See first batch
#batch = next(iter(train_loader))
#print(batch[0]) # 0 -> input_ids , 1 -> attention_masks, 2 -> type_ids, 3 -> targets
#print(train_loader.batch_size) #tamaño del batch
print('Number of train batches: {}'.format(len(train_loader)))# número de batches
print('Number of evaluation batches: {}'.format(len(val_loader)))

Number of train batches: 180
Number of evaluation batches: 47


In [ ]:
class BertBaseUncasedRegressor(nn.Module):
  def __init__(self, num_classes = 1, dropout=0.1, attentions=True, n_layers = 0):
      super().__init__()
      self.bert = BertModel.from_pretrained(BERT_PATH, output_attentions=attentions) #load the model
      # Return:
      # last_hidden_state.shape -> [batch_size, num_tokens_in_sequence,hidden_size] (bert_base hidden_size = 768)
      # pooler_output.shape -> [batch_size, hidden_size] se utiliza una capa de pooling simple que aplica una transformación
      # lineal seguida de una función de activación tangente hiperbólica (tanh) a la última representación oculta (last_hidden_state) del token [CLS].
      layers = []
      for _ in range(n_layers):
          layers.append(nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size)) #self.bert.config.hidden_size -> nos da el tamaño oculto
          layers.append(nn.Dropout(dropout))
      layers.append(nn.Linear(self.bert.config.hidden_size, num_classes)) #self.bert.config.hidden_size -> nos da el tamaño oculto

      self.regressor = nn.Sequential(*layers)


  def forward(self, input_ids, type_ids, mask):
      bert_output =self.bert(input_ids=input_ids, token_type_ids = type_ids, attention_mask= mask)
      attention_mask = mask.unsqueeze(-1).expand(bert_output.last_hidden_state.shape).float()
      embeddings = bert_output.last_hidden_state * attention_mask
      embeddings_sum = torch.sum(embeddings, dim=1)
      counts = torch.clamp(attention_mask.sum(1), min=1e-9)
      output_mean_pooling = embeddings_sum / counts
      #output = bert_output.pooler_output
      #output = bert_output.last_hidden_state[:, 0, :]
      logits = self.regressor(output_mean_pooling)
      #logits = self.regressor(output_drop)
      return logits, bert_output

In [ ]:
pp = torch.tensor([[[1,2,3],[4,5,6]],[[11,12,13],[14,15,16]]])
pp2 = torch.tensor([[[1,1,1],[1,1,0]],[[1,1,1],[1,0,0]]])
ma = pp*pp2
sum = torch.sum(ma, dim=1)
print(ma)
print(ma.shape)
print(sum)
print(sum.shape)
counts = torch.clamp(pp2.sum(1), min=1e-9)
print(counts)
print(sum/counts)

In [ ]:
"""model = BertBaseUncasedRegressor().to(device)"""

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
"""
criterion = nn.MSELoss()
LEARNING_RATE = 0.0001
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
"""

In [ ]:
def pearson_corr(y_pred, y_true):
    all_preds = np.concatenate(y_pred)
    all_targets = np.concatenate(y_true)
    return np.corrcoef(all_preds, all_targets)[0, 1]

In [ ]:
def spearman_corr(y_pred, y_true):
    all_preds = np.concatenate(y_pred)
    all_targets = np.concatenate(y_true)

    corr, _ = spearmanr(all_preds, all_targets)
    return corr

In [ ]:
def train_loop(model,loader,optimizer,criterion,device,clip = 1.0):
    #Training loop
    model.train()
    loss_sum = 0
    all_preds = []
    all_targets = []
    seed = 42
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    for i, batch in enumerate(loader):

        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_type_ids = batch[2].to(device)
        labels = batch[3].to(device)

        optimizer.zero_grad()
        #Forward
        outputs, _ = model(input_ids=b_input_ids, type_ids = b_type_ids, mask = b_attention_mask)
        outputs = outputs.squeeze(-1)
        #Loss
        loss = criterion(outputs.view(-1), labels.float())
        #Backprop
        loss.backward()
        #prevent gradients from exploding
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        #Update params
        optimizer.step()

        loss_sum += loss.item()

        all_preds.append(outputs.detach().cpu().numpy())
        all_targets.append(labels.float().detach().cpu().numpy())

    epoch_train_loss = loss_sum / len(loader)
    epoch_train_pearson = pearson_corr(all_preds, all_targets)
    epoch_train_spearman = spearman_corr(all_preds, all_targets)

    return epoch_train_loss, epoch_train_pearson, epoch_train_spearman


In [ ]:
def evaluation_loop(model,loader,criterion,device):
    #Evaluation loop
    seed = 42
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    model.eval()
    with torch.no_grad():
        loss_sum = 0
        all_preds = []
        all_targets = []

        for i, batch in enumerate(loader):
            b_input_ids = batch[0].to(device)
            b_attention_mask = batch[1].to(device)
            b_type_ids = batch[2].to(device)
            labels = batch[3].to(device)

            #Forward
            outputs, _ = model(input_ids=b_input_ids, type_ids = b_type_ids, mask = b_attention_mask)
            outputs = outputs.squeeze(-1)
            #Loss
            loss = criterion(outputs.view(-1), labels.float())

            loss_sum += loss.item()

            all_preds.append(outputs.detach().cpu().numpy())
            all_targets.append(labels.float().detach().cpu().numpy())

        epoch_dev_loss = loss_sum / len(loader)
        epoch_dev_pearson = pearson_corr(all_preds, all_targets)
        epoch_dev_spearman = spearman_corr(all_preds, all_targets)

    return epoch_dev_loss, epoch_dev_pearson, epoch_dev_spearman

In [ ]:
def save_metrics_dataframe(metrics_dict, study_name, path_base):
  # Abrir archivo en modo escritura y especificar el separador de campos
  path = path_base + 'metrics_' + study_name + ".csv"
  with open(path, "a+", newline="") as f:
      metrics = [metrics_dict]
      # Crear objeto escritor CSV y especificar el separador de campos
      metrics_csv = csv.DictWriter(f, fieldnames=['train_loss','valid_loss','spearman_train','spearman_val','pearson_train','pearson_val'],delimiter=",")

      # Verificar si se ha escrito el encabezado del archivo
      if f.tell() == 0:
        metrics_csv.writeheader()

      # Escribir cada fila de datos
      for metric in metrics:
          metrics_csv.writerow(metric)

In [ ]:
study_name = 'study-bert-mean-pooling-base'

In [ ]:
def isNan(value):
  return 0 if math.isnan(value) else value

In [ ]:
def objective(trial):
  params = {
      "num_layers": trial.suggest_int("num_layer", 0, 3),
      "dropout": trial.suggest_uniform("dropout", 0, 0.7),
      "optimizer": trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
      "learning_rate": trial.suggest_loguniform("learning_rate",1e-5, 1e-4)
  }

  model = BertBaseUncasedRegressor(dropout=params["dropout"], n_layers = params["num_layers"]).to(device)
  criterion = nn.MSELoss()
  optimizer = getattr(optim, params["optimizer"])(model.parameters(), lr=params["learning_rate"])

  NUM_EPOCHS = 4
  best_valid_loss = float('inf')
  MODEL_FILE_NAME = CORPUS+'_'+BERT_PATH+'_'+str(MAX_LEN)+'_tunned_model.pt'
  history = {"train": {"loss": []}, "test": {"loss": []}}

  torch.cuda.empty_cache()
  seed = 42
  np.random.seed(seed)
  random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)

  train_loss_list, val_loss_list, spearman_train, spearman_val, pearson_train, pearson_val = [], [], [], [], [], []

  for epoch in range(NUM_EPOCHS):

    start_time = time.time()

    epoch_train_loss, epoch_train_pearson, epoch_train_spearman = train_loop(model,train_loader,optimizer,criterion,device)
    epoch_dev_loss, epoch_dev_pearson, epoch_dev_spearman = evaluation_loop(model,val_loader,criterion,device)

    elapsed_time = time.time() - start_time

    if trial.should_prune():
      raise optuna.exceptions.TrialPruned()

    #nos quedamos con el modelo que tiene mejor pérdida de validación
    if epoch_dev_loss < best_valid_loss:
      best_valid_loss = epoch_dev_loss
      torch.save(model.state_dict(), MODEL_FILE_NAME)

    train_loss_list.append(isNan(epoch_train_loss))
    val_loss_list.append(isNan(epoch_dev_loss))
    spearman_train.append(isNan(epoch_train_spearman))
    spearman_val.append(isNan(epoch_dev_spearman))
    pearson_train.append(isNan(epoch_train_pearson))
    pearson_val.append(isNan(epoch_dev_pearson))

    history["train"]["loss"].append(epoch_train_loss)
    history["test"]["loss"].append(epoch_dev_loss)

    print('-' * 80)
    print(f'Epoch: {epoch+1:03}/{NUM_EPOCHS} | Time: {elapsed_time:.4f}s | Train loss: {epoch_train_loss:.4f} | Dev loss: {epoch_dev_loss:.4f}')
    print(f'Train Pearson Coef: {epoch_train_pearson:.4f} | Dev Pearson Coef: {epoch_dev_pearson:.4f}')
    print(f'Train Spearman Coef: {epoch_train_spearman:.4f} | Dev Spearman Coef: {epoch_dev_spearman:.4f}')

  metrics_dict = {
      'train_loss' : train_loss_list,
      'valid_loss' : val_loss_list,
      'spearman_train': spearman_train,
      'spearman_val': spearman_val,
      'pearson_train': pearson_train,
      'pearson_val': pearson_val
  }

  save_metrics_dataframe(metrics_dict, study_name, path_base)

  return best_valid_loss

In [ ]:
#obtener ip de colab para dar acceso a la DB
!curl ipecho.net/plain

34.32.145.133

In [ ]:
# Define the connection URL.
url = "postgresql://{user}:{password}@{host}:{port}/{database}"
url = url.format(
    user='postgres',
    password="password",
    host='35.184.132.46',
    port='5432',
    database='stsb-base-mean-pooling',
)

# Create the engine and the database (if it doesn't exist yet).
engine = create_engine(url)


# Define the storage.
storage = RDBStorage(url)

In [ ]:
# Define el estudio de Optuna utilizando el storage
study = optuna.create_study(study_name=study_name, direction="minimize", storage=storage, load_if_exists=True)

[I 2023-05-08 17:40:07,526] Using an existing study with name 'study-bert-mean-pooling-base' instead of creating a new one.


In [ ]:
study.optimize(objective, n_trials=10)

<ipython-input-27-53008359ab4f>:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "dropout": trial.suggest_uniform("dropout", 0, 0.7),
<ipython-input-27-53008359ab4f>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "learning_rate": trial.suggest_loguniform("learning_rate",1e-5, 1e-4)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


--------------------------------------------------------------------------------
Epoch: 001/4 | Time: 116.1788s | Train loss: 0.9328 | Dev loss: 0.4845
Train Pearson Coef: 0.7551 | Dev Pearson Coef: 0.8863
Train Spearman Coef: 0.7245 | Dev Spearman Coef: 0.8821
--------------------------------------------------------------------------------
Epoch: 002/4 | Time: 120.6032s | Train loss: 0.3254 | Dev loss: 0.6899
Train Pearson Coef: 0.9210 | Dev Pearson Coef: 0.8894
Train Spearman Coef: 0.9049 | Dev Spearman Coef: 0.8846
--------------------------------------------------------------------------------
Epoch: 003/4 | Time: 123.7579s | Train loss: 0.1764 | Dev loss: 0.4725
Train Pearson Coef: 0.9581 | Dev Pearson Coef: 0.8961
Train Spearman Coef: 0.9509 | Dev Spearman Coef: 0.8912
--------------------------------------------------------------------------------
Epoch: 004/4 | Time: 125.5288s | Train loss: 0.1193 | Dev loss: 0.4521
Train Pearson Coef: 0.9719 | Dev Pearson Coef: 0.8945
Train Sp

[I 2023-05-08 17:48:51,986] Trial 141 finished with value: 0.45209156103590703 and parameters: {'num_layer': 1, 'dropout': 0.0774883090038255, 'optimizer': 'RMSprop', 'learning_rate': 2.870329876059525e-05}. Best is trial 131 with value: 0.4364643043026011.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly iden

--------------------------------------------------------------------------------
Epoch: 001/4 | Time: 123.6297s | Train loss: 1.3436 | Dev loss: 0.7334
Train Pearson Coef: 0.6223 | Dev Pearson Coef: 0.8531
Train Spearman Coef: 0.5922 | Dev Spearman Coef: 0.8529
--------------------------------------------------------------------------------
Epoch: 002/4 | Time: 125.0165s | Train loss: 0.5752 | Dev loss: 0.7016
Train Pearson Coef: 0.8555 | Dev Pearson Coef: 0.8668
Train Spearman Coef: 0.8293 | Dev Spearman Coef: 0.8641
--------------------------------------------------------------------------------
Epoch: 003/4 | Time: 125.8772s | Train loss: 0.3653 | Dev loss: 0.5874
Train Pearson Coef: 0.9110 | Dev Pearson Coef: 0.8737
Train Spearman Coef: 0.8944 | Dev Spearman Coef: 0.8681
--------------------------------------------------------------------------------
Epoch: 004/4 | Time: 125.5742s | Train loss: 0.2653 | Dev loss: 0.5753
Train Pearson Coef: 0.9361 | Dev Pearson Coef: 0.8751
Train Sp

...

[I 2023-05-08 18:49:24,385] Trial 148 finished with value: 0.49102906565716925 and parameters: {'num_layer': 1, 'dropout': 0.1417849176794103, 'optimizer': 'RMSprop', 'learning_rate': 2.274083723156822e-05}. Best is trial 131 with value: 0.4364643043026011.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly iden

--------------------------------------------------------------------------------
Epoch: 001/4 | Time: 126.2698s | Train loss: 0.8168 | Dev loss: 0.5414
Train Pearson Coef: 0.7881 | Dev Pearson Coef: 0.8854
Train Spearman Coef: 0.7570 | Dev Spearman Coef: 0.8822
--------------------------------------------------------------------------------
Epoch: 002/4 | Time: 125.2183s | Train loss: 0.2604 | Dev loss: 0.5830
Train Pearson Coef: 0.9373 | Dev Pearson Coef: 0.8899
Train Spearman Coef: 0.9231 | Dev Spearman Coef: 0.8861
--------------------------------------------------------------------------------
Epoch: 003/4 | Time: 125.4350s | Train loss: 0.1293 | Dev loss: 0.4785
Train Pearson Coef: 0.9694 | Dev Pearson Coef: 0.8890
Train Spearman Coef: 0.9628 | Dev Spearman Coef: 0.8854
--------------------------------------------------------------------------------
Epoch: 004/4 | Time: 125.5662s | Train loss: 0.1057 | Dev loss: 0.5214
Train Pearson Coef: 0.9752 | Dev Pearson Coef: 0.8936
Train Sp

[I 2023-05-08 18:58:02,853] Trial 149 finished with value: 0.47847019770043964 and parameters: {'num_layer': 1, 'dropout': 0.0998867219505796, 'optimizer': 'RMSprop', 'learning_rate': 2.1268523118731523e-05}. Best is trial 131 with value: 0.4364643043026011.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly ide

--------------------------------------------------------------------------------
Epoch: 001/4 | Time: 126.1524s | Train loss: 0.8125 | Dev loss: 0.5826
Train Pearson Coef: 0.7899 | Dev Pearson Coef: 0.8889
Train Spearman Coef: 0.7589 | Dev Spearman Coef: 0.8865
--------------------------------------------------------------------------------
Epoch: 002/4 | Time: 125.1313s | Train loss: 0.2526 | Dev loss: 0.4813
Train Pearson Coef: 0.9393 | Dev Pearson Coef: 0.8968
Train Spearman Coef: 0.9262 | Dev Spearman Coef: 0.8931
--------------------------------------------------------------------------------
Epoch: 003/4 | Time: 125.4868s | Train loss: 0.1378 | Dev loss: 0.4490
Train Pearson Coef: 0.9674 | Dev Pearson Coef: 0.8951
Train Spearman Coef: 0.9610 | Dev Spearman Coef: 0.8914
--------------------------------------------------------------------------------
Epoch: 004/4 | Time: 125.9507s | Train loss: 0.0993 | Dev loss: 0.6022
Train Pearson Coef: 0.9766 | Dev Pearson Coef: 0.8944
Train Sp

[I 2023-05-08 19:06:42,602] Trial 150 finished with value: 0.4489823116901073 and parameters: {'num_layer': 1, 'dropout': 0.10825515065611765, 'optimizer': 'RMSprop', 'learning_rate': 2.509632906548691e-05}. Best is trial 131 with value: 0.4364643043026011.


In [ ]:
df_optuna = study.trials_dataframe(attrs=("number", "value", "params", "state")).dropna(subset=['value']) #eliminar valores NaN
# Leer el archivo metrics.CSV y crear un DataFrame
path_metrics = path_base + 'metrics_' + study_name + ".csv"
df_metrics = pd.read_csv(path_metrics).dropna(subset=['train_loss'])
print(len(df_optuna))
print(len(df_metrics ))

150
10


In [ ]:
#df_metrics = df_metrics[['train_loss', 'valid_loss', 'pearson_train', 'pearson_val', 'spearman_train', 'spearman_val']]

In [ ]:
#df_metrics = df_metrics.rename(columns={'train_loss': 'train_loss', 'valid_loss': 'val_loss', 'pearson_train': 'Pearson_train','pearson_val': 'Pearson_val', 'spearman_train': 'Spearman_train', 'spearman_val': 'Spearman_val'})

In [ ]:
df_metrics

In [ ]:
df_old = pd.read_csv(path_base + study_name + "_COMPLETE_SORT.csv")
print("Old trials: ", len(df_old))

Old trials:  140


In [ ]:
#df_old = df_old[['number','value','params_dropout','params_learning_rate','params_num_layer','params_optimizer','state','train_loss', 'val_loss', 'Spearman_train','Spearman_val','Pearson_train','Pearson_val']]

In [ ]:
#df_old = df_old.rename(columns={'val_loss': 'valid_loss', 'Spearman_train': 'spearman_train', 'Spearman_val': 'spearman_val', 'Pearson_train':'pearson_train','Pearson_val':'pearson_val'})


In [ ]:
# Seleccionar los valores de df_optuna que no están ya en df_old
if len(df_optuna)-len(df_metrics) == len(df_old):
    df_optuna_filter = df_optuna.tail(len(df_metrics)).reset_index(drop=True)
else:
    # Seleccionar los valores de df_optuna que no están ya en df_old
    mask = ~df_optuna['value'].isin(df_old['value'])
    df_optuna_filter = df_optuna[mask].reset_index(drop=True)
    print(len(df_optuna_filter))

In [ ]:
def concat(df_optuna_filter, df_metrics, df_old):
  # Unir dataframe de metrics con dataframe de optuna de forma horizontal y manejar los valores inexistentes
  df_join = pd.concat([df_optuna_filter, df_metrics], axis=1, sort=False)
  # Unir dataframe de metrics con dataframe de optuna de forma vertical y manejar los valores inexistentes
  df_complete_updated = pd.concat([df_old, df_join], axis=0, sort=False).drop_duplicates()
  print("All trials: ", len(df_complete_updated))
  assert len(df_join) + len(df_old) == len(df_complete_updated)
  return df_complete_updated

In [ ]:
if len(df_optuna_filter) == len(df_metrics):
  df_complete_updated = concat(df_optuna_filter, df_metrics, df_old).dropna(subset=['train_loss', 'value'])

if len(df_optuna_filter) > len(df_metrics):
  num_rows_to_add = len(df_optuna_filter) - len(df_metrics)
  new_rows = [[np.nan] * len(df_metrics.columns) for i in range(num_rows_to_add)]
  new_df = pd.DataFrame(new_rows, columns=df_metrics.columns)
  df_metrics = pd.concat([new_df, df_metrics], ignore_index=True)
  df_complete_updated = concat(df_optuna_filter, df_metrics, df_old).dropna(subset=['train_loss','value'])

All trials:  150


In [ ]:
df_complete_updated

In [ ]:
# Ordenar el DataFrame unido por valor
df = df_complete_updated.sort_values(by='value', ascending=True)
# Obtener la prueba con el mejor valor
best_trial = df.iloc[0]
print("Best trial: ")
print(best_trial)

Best trial: 
number                                                                131
value                                                            0.436464
params_dropout                                                   0.137565
params_learning_rate                                             0.000026
params_num_layer                                                        1
params_optimizer                                                  RMSprop
state                                                            COMPLETE
train_loss              [0.8522996495167414, 0.27165179416123364, 0.14...
valid_loss              [0.4935647572608704, 0.6157969382215054, 0.485...
spearman_train          [0.7493913309890617, 0.9201128253859435, 0.958...
spearman_val            [0.8887677638036555, 0.8914520592480365, 0.890...
pearson_train           [0.7780476503426055, 0.9345250195924778, 0.965...
pearson_val             [0.8917265452021073, 0.8954647163559462, 0.894...
Name: 0, dtype: object


In [ ]:
#Guardar CSV completo y ordenado
# ruta de acceso a la carpeta de Google Drive montada
os.remove(path_base + study_name + "_COMPLETE_SORT.csv")
path_ = path_base + study_name + "_COMPLETE_SORT.csv"
# guardar el archivo CSV en Google Drive
df.to_csv(path_, index=False)
files.download(path_)
os.remove(path_metrics)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>